# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Imports Libraries</p></div>


In [ ]:
# =======================================
# PART 1: Configuration & Imports
# =======================================
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

# Paths & Hyperparameters
IMG_SIZE = 256  # You can try 512 for better score if memory allows
BATCH_SIZE = 8
EPOCHS = 30  # Increase epochs for better learning
LR = 1e-4
DATA_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/"

# Threshold tuning for F1/Dice
THRESHOLD = 0.35


In [ ]:
# =======================================
# PART 2: Dataset + Augmentation
# =======================================
!pip install albumentations --quiet
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 1️⃣ Augmentation pipelines
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.GaussianBlur(blur_limit=(3,5), p=0.3),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

mask_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    ToTensorV2()
])

# 2️⃣ Dataset class
class ForgeryDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, is_authentic=None, transforms=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.is_authentic = is_authentic
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        
        if self.is_authentic[idx]:
            mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        else:
            mask = np.array(np.load(self.mask_paths[idx]))
            if mask.ndim == 3:
                mask = mask[:, :, 0]
            mask = (mask > 0).astype(np.uint8)
        
        if self.transforms:
            augmented = self.transforms(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        
        return img, mask

# 3️⃣ Load file paths
forged_images = sorted(glob(os.path.join(DATA_PATH, 'train_images/forged/*.png')))
train_masks = sorted(glob(os.path.join(DATA_PATH, 'train_masks/*.npy')))
authentic_images = sorted(glob(os.path.join(DATA_PATH, 'train_images/authentic/*.png')))

matched_forged_images, matched_masks = [], []
for img_path in forged_images:
    img_name = os.path.basename(img_path).replace('.png','')
    mask_path = os.path.join(DATA_PATH, f'train_masks/{img_name}.npy')
    if os.path.exists(mask_path):
        matched_forged_images.append(img_path)
        matched_masks.append(mask_path)

all_images = matched_forged_images + authentic_images
all_masks = matched_masks + [None]*len(authentic_images)
all_is_authentic = [False]*len(matched_forged_images) + [True]*len(authentic_images)

# 4️⃣ Train-validation split
from sklearn.model_selection import train_test_split

train_imgs, val_imgs, train_masks_split, val_masks_split, train_auth, val_auth = train_test_split(
    all_images, all_masks, all_is_authentic, test_size=0.2, random_state=SEED, stratify=all_is_authentic
)

# 5️⃣ Create Datasets and Dataloaders
train_dataset = ForgeryDataset(train_imgs, train_masks_split, train_auth, transforms=train_transform)
val_dataset = ForgeryDataset(val_imgs, val_masks_split, val_auth, transforms=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")


In [ ]:
# =======================================
# PART 3: Model + Loss + Optimizer + Training Loop
# =======================================

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# 1️⃣ UNet model (simplified)
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64,128,256,512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(2,2)
        
        # Down part
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
        
        # Up part
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))
        
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
    
    def forward(self, x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            if x.shape != skip_connection.shape:
                x = F.interpolate(x, size=skip_connection.shape[2:])
            x = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](x)
        
        return self.final_conv(x)

# 2️⃣ Loss functions
class DiceBCELoss(nn.Module):
    def __init__(self, smooth=1):
        super().__init__()
        self.smooth = smooth
        self.bce = nn.BCEWithLogitsLoss()
    
    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        intersection = (preds * targets).sum(dim=(2,3))
        dice = (2 * intersection + self.smooth) / (preds.sum(dim=(2,3)) + targets.sum(dim=(2,3)) + self.smooth)
        dice_loss = 1 - dice.mean()
        bce_loss = self.bce(preds, targets)
        return bce_loss + dice_loss

# 3️⃣ Initialize model, optimizer, loss
model = UNet(in_channels=3, out_channels=1).to(DEVICE)
criterion = DiceBCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# 4️⃣ Training function
from tqdm import tqdm

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=10):
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            imgs = imgs.to(DEVICE)
            masks = masks.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * imgs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                imgs = imgs.to(DEVICE)
                masks = masks.to(DEVICE)
                outputs = model(imgs)
                loss = criterion(outputs, masks)
                val_loss += loss.item() * imgs.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        if scheduler:
            scheduler.step(val_loss)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
            print("✅ Saved Best Model!")
    
    return model

# 5️⃣ Start training
# trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20)


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
# Import CV2 (used by albumentations internally for image operations)
import cv2 
import torch
from torch.utils.data import Dataset, DataLoader
# Import albumentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# =======================================
# Configuration Variables
# =======================================
IMG_SIZE = 256
BATCH_SIZE = 16
DATA_DIR = "./data" # Adjust this path
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume train_df is loaded here:
# train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv")) 


In [ ]:
train_transforms = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE, interpolation=cv2.INTER_NEAREST), # Resize first
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(), # Converts numpy array to torch tensor
])

val_transforms = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE, interpolation=cv2.INTER_NEAREST), # Resize first
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# For masks that don't need normalization/color jitter, the main transforms suffice 
# because ToTensorV2 handles masks correctly (no normalization applied to them).


In [ ]:
class ForgeryDataset(Dataset):
    # Rename 'transform' to 'transforms' for clarity with albumentations naming
    def __init__(self, df, img_dir, mask_dir, transforms=None): 
        self.df = df
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transforms = transforms # Now holds the A.Compose object

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["image_name"]
        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace(".jpg", ".png"))

        # Open image and mask using PIL
        image = Image.open(img_path).convert("RGB")
        
        # Check if mask exists, otherwise assume authentic (zero mask)
        if os.path.exists(mask_path):
            mask = Image.open(mask_path).convert("L")
        else:
            # Create a zero mask matching the image size/mode if authentic
            mask = Image.fromarray(np.zeros((image.height, image.width), dtype=np.uint8))
        
        # Convert PIL images to numpy arrays, which Albumentations expects
        image = np.array(image)
        mask = np.array(mask)
        
        if self.transforms:
            # This is where the magic happens: albumentations syncs transforms
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            # Ensure mask tensor is float and has the correct dimensions (e.g. 1 channel)
            if len(mask.shape) == 2:
                mask = mask.unsqueeze(0).float()
            mask = mask.float() / 255.0 # Normalize mask values to 0.0 or 1.0

        return image, mask

# Train-validation split (Requires 'train_df' to be defined)
train_split, val_split = train_test_split(train_df, test_size=0.2, random_state=42)

# Initialize datasets with the single 'transforms' argument
train_dataset = ForgeryDataset(train_split, os.path.join(DATA_DIR, "train_images"),
                               os.path.join(DATA_DIR, "train_masks"), train_transforms)
val_dataset = ForgeryDataset(val_split, os.path.join(DATA_DIR, "train_images"),
                             os.path.join(DATA_DIR, "train_masks"), val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [ ]:
# Run this in a notebook cell to install the library
!pip install segmentation-models-pytorch
!pip install timm # segmentation_models_pytorch often needs timm as a dependency


In [ ]:
# =======================================
# Training and Validation Functions
# =======================================

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for imgs, masks in tqdm(loader, desc="Training"):
        imgs, masks = imgs.to(device), masks.to(device)

        # Forward pass
        preds = model(imgs)
        loss = criterion(preds, masks)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
    
    return running_loss / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for imgs, masks in tqdm(loader, desc="Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            loss = criterion(preds, masks)
            running_loss += loss.item() * imgs.size(0)
            
    return running_loss / len(loader.dataset)



In [ ]:
DATA_DIR = "./data" 
os.path.join(DATA_DIR, "train_images") # Result: './data/train_images'


In [ ]:
# Change the previous line where you defined DATA_DIR
DATA_DIR = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"

# Now the script will look here:
# /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images
# /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks


In [ ]:
# Use a single dot to mean "current directory"
DATA_DIR = "."


In [ ]:
import os
for root, dirs, files in os.walk("/"):
    if "train_images" in dirs:
        print(f"Found train_images folder at: {os.path.join(root, 'train_images')}")
        # Stop searching if found one
        break 


In [ ]:
# =======================================
# Configuration Variables and Imports (Top of your script)
# =======================================

# Replace the previous DATA_DIR = "./data" with the actual path you found:

DATA_DIR = "/kaggle/input/recodai-luc-scientific-image-forgery-detection" 
# OR
# DATA_DIR = "C:/Users/YourName/Desktop/KaggleForgeries" 
# OR
# DATA_DIR = "/home/username/data/forgery_detection"

# ... (rest of your configuration code remains the same) ...


In [ ]:
# Code from inside the ForgeryDataset class
def __getitem__(self, idx):
    # ...
    augmented = self.transforms(image=image, mask=mask) # Uses 'self.transforms'
    # ...


In [ ]:
# The problematic line, if you copied it exactly:
test_transform = val_transforms # It expects val_transforms to be available globally

# ...
# augmented = self.transforms(image=input_image_np) # If this is in your submission code, it's wrong.


In [ ]:
# Make sure these are imported at the top of your notebook:
# import albumentations as A
# import cv2
# from albumentations.pytorch.transforms import ToTensorV2
# import numpy as np
# from PIL import Image

def create_submission(model, test_dir, submission_path, img_size):
    model.eval()
    test_images_names = os.listdir(test_dir)
    submission_data = []

    # Define the transformation needed for testing locally within the function
    # so it doesn't need to reference 'self' or global 'val_transforms'
    test_transform = A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=cv2.INTER_NEAREST),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

    with torch.no_grad():
        for img_name in tqdm(test_images_names, desc="Predicting Test Images"):
            img_path = os.path.join(test_dir, img_name)
            image = Image.open(img_path).convert("RGB")
            original_shape = image.size # (width, height)
            
            # Convert to numpy for albumentations, then transform
            input_image_np = np.array(image)
            # Use the local variable 'test_transform'
            augmented = test_transform(image=input_image_np) 
            input_image = augmented['image'].unsqueeze(0).to(DEVICE)
            
            # Get model output
            output = model(input_image)
            
            # Post-process: sigmoid and threshold
            mask_pred = torch.sigmoid(output).cpu().numpy().squeeze()
            
            # Resize the mask back to original image size using PIL
            if mask_pred.ndim == 3 and mask_pred.shape == 1:
                mask_pred = mask_pred.squeeze(0)
                
            mask_pred_resized = Image.fromarray((mask_pred * 255).astype(np.uint8)).resize(original_shape, Image.NEAREST)
            mask_pred_resized_np = np.array(mask_pred_resized) > 127 # Binary mask (True/False)
            
            # RLE Encode (ensure rle_encode is defined globally)
            rle_mask = rle_encode(mask_pred_resized_np)
            
            submission_data.append({"case_id": img_name.replace('.jpg', ''), "annotation": rle_mask})

    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(submission_path, index=False)
    print(f"Submission file saved to {submission_path}")

# Run the submission function by passing IMG_SIZE as an argument
create_submission(
    model, 
    os.path.join(DATA_DIR, "test_images"), 
    "submission.csv",
    IMG_SIZE # Pass the global IMG_SIZE variable
)


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SmallUNet().to(DEVICE)


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for imgs, masks in train_loader:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}")


In [ ]:
# =======================================
# Prediction and Submission
# =======================================

def create_submission(model, test_dir, sample_sub_path, submission_path):
    model.eval()
    test_images_names = os.listdir(test_dir)
    submission_data = []

    # Use the same validation transform for test images (without augmentation)
    test_transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    with torch.no_grad():
        for img_name in tqdm(test_images_names):
            img_path = os.path.join(test_dir, img_name)
            image = Image.open(img_path).convert("RGB")
            original_shape = image.size # (width, height)
            
            input_image = test_transform(image).unsqueeze(0).to(DEVICE)
            output = model(input_image)
            
            # Post-process the model output
            # Assuming U-Net output (logits), apply sigmoid and threshold
            mask_pred = torch.sigmoid(output).cpu().numpy().squeeze()
            # Resize the mask back to original image size
            mask_pred_resized = Image.fromarray((mask_pred * 255).astype(np.uint8)).resize(original_shape, Image.NEAREST)
            mask_pred_resized_np = np.array(mask_pred_resized) > 127 # Binary mask
            
            rle_mask = rle_encode(mask_pred_resized_np)
            
            submission_data.append({"case_id": img_name, "annotation": rle_mask})

    submission_df = pd.DataFrame(submission_data)
    # The competition expects "case_id" without extension for submission, check sample_submission.csv format
    submission_df['case_id'] = submission_df['case_id'].str.replace('.jpg', '', regex=False) 
    submission_df.to_csv(submission_path, index=False)
    print(f"Submission file saved to {submission_path}")

# Example usage (needs a working 'model' variable):
create_submission(model, os.path.join(DATA_DIR, "test_images"), 
                  os.path.join(DATA_DIR, "sample_submission.csv"), 
                  "submission.csv")


In [ ]:
MODEL_PATH = "./best_model.pth" 
